In [59]:
import duckdb

In [60]:
con = duckdb.connect("database_for_testing_duck.duckdb")

In [61]:
con.execute("""
    CREATE TABLE IF NOT EXISTS bank AS 
    SELECT * FROM read_csv('bank_information.csv')
""")
con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,database_for_testing_duck,main,bank,"[index, age, job, marital, education, default,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, VA...",False


In [62]:
con.execute("SELECT * FROM bank WHERE duration < 100 LIMIT 5").fetchdf()

,index,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,9,25,services,single,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,None,None,None,4.857,None,False
1,10,41,blue-collar,married,unknown,unknown,no,no,telephone,may,...,1,999,0,nonexistent,None,None,None,4.857,None,False
2,20,30,unemployed,married,high.school,no,no,no,telephone,may,...,1,999,0,nonexistent,None,None,None,4.857,None,False
3,25,35,technician,married,university.degree,no,no,yes,telephone,may,...,1,999,0,nonexistent,None,None,None,4.857,None,False
4,26,59,technician,married,unknown,no,yes,no,telephone,may,...,1,999,0,nonexistent,None,None,None,4.857,None,False


In [63]:
rel = con.table("bank")
rel.columns

['index',
 'age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [64]:
rel.filter("duration < 100").project("job,education,loan").order("job").limit(3).df()

,job,education,loan
0,blue-collar,unknown,no
1,blue-collar,unknown,no
2,blue-collar,basic.9y,no


In [65]:
res = duckdb.query("""SELECT 
                            job,
                            COUNT(*) AS total_clients_contacted,
                            AVG(duration) AS avg_campaign_duration,
                        FROM 
                            'bank_information.csv'
                        WHERE 
                            age > 30
                        GROUP BY 
                            job
                        ORDER BY 
                            total_clients_contacted DESC;""")
res.df()

,job,total_clients_contacted,avg_campaign_duration
0,admin.,26,245.807692
1,blue-collar,23,320.695652
2,technician,13,385.153846
3,management,9,283.444444
4,services,8,323.750000
5,unknown,4,239.250000
6,housemaid,3,273.333333
7,entrepreneur,3,601.666667
8,retired,2,258.000000
9,unemployed,2,345.000000


In [66]:
con.close()

In [67]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.core import StorageContext

from IPython.display import Markdown, display

In [68]:
import os
from llama_index.llms.openai import OpenAI
import config

config.apiToken()

llm = OpenAI(model="gpt-4o",api_key=os.environ["OPENAI_API_KEY"])

In [69]:
from llama_index.embeddings.openai import OpenAIEmbedding
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
)

In [70]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [71]:
documents = SimpleDirectoryReader("Data").load_data()

In [72]:
vector_store = DuckDBVectorStore(database_name = "database_for_testing_duck.duckdb",table_name = "blog",persist_dir="./", embed_dim=1536)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

In [74]:
con = duckdb.connect("database_for_testing_duck.duckdb")

con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,database_for_testing_duck,main,bank,"[index, age, job, marital, education, default,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, VA...",False
1,database_for_testing_duck,main,blog,"[node_id, text, embedding, metadata_]","[VARCHAR, VARCHAR, FLOAT[1536], JSON]",False


In [75]:
query_engine = index.as_query_engine()
response = query_engine.query("Who wrote 'DuckDB Tutorial: Building AI Projects'?")
display(Markdown(f"<b>{response}</b>"))

<b>The 'DuckDB Tutorial: Building AI Projects' was written by Abid Ali Awan.</b>

In [76]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm
)

response = chat_engine.chat(
    "How do I implement DuckDB with LLM? Provide step by step instructions"
)

display(Markdown(response.response))

To implement DuckDB with a Large Language Model (LLM), you can follow these steps, which are based on building a Retrieval-Augmented Generation (RAG) application using DuckDB as a vector database and retriever:

1. **Install Necessary Packages**: 
   - First, ensure you have Python installed on your system. Then, install the required packages using pip. You will need DuckDB, LlamaIndex, and LlamaIndex-Vector-Stores-DuckDB. You can install them using the following commands:
     ```bash
     %pip install duckdb
     %pip install llama-index
     %pip install llama-index-vector-stores-duckdb
     ```

2. **Set Up Your Environment**:
   - Import the necessary Python packages in your script. This will typically include DuckDB and any other libraries you need for your specific application.

3. **Create and Retrieve the Index**:
   - Use LlamaIndex to create an index that will be stored in DuckDB. This involves setting up your data in a format that can be indexed and then using LlamaIndex functions to create the index within DuckDB.

4. **Execute SQL Queries**:
   - Use DuckDB's query function to execute SQL queries on your data. For example, you might want to retrieve specific information or perform aggregations. Here's a sample query that finds job titles of clients over the age of 30, counts the number of clients contacted for each job, and calculates the average duration of the campaign:
     ```python
     res = duckdb.query("""
     SELECT 
         job,
         COUNT(*) AS total_clients_contacted,
         AVG(duration) AS avg_campaign_duration
     FROM 
         'bank-marketing.csv'
     WHERE 
         age > 30
     GROUP BY 
         job
     ORDER BY 
         total_clients_contacted DESC;
     """)
     df = res.df()
     ```

5. **Close the Connection**:
   - After executing your queries and retrieving the necessary data, make sure to close the connection to the DuckDB database to release any resources and prevent potential memory leaks:
     ```python
     con.close()
     ```

These steps provide a basic framework for integrating DuckDB with an LLM using a RAG approach. You can expand upon this by incorporating more complex queries, additional data processing, and integrating the results with your LLM for further analysis or generation tasks.

In [77]:
response = chat_engine.chat(
    "Could you please provide more details about the integration part with the LLM and how I create a memory buffer?"
)
display(Markdown(response.response))

To integrate DuckDB with a Large Language Model (LLM) and create a memory buffer, you can follow these steps:

1. **Set Up the LLM**:
   - Use the OpenAI API to set up your LLM. For this example, we'll use the GPT-4o model. You need to provide the model name and your API key. If your environment variable is set with the name “OPENAI_API_KEY,” you can omit the API key in the code.
     ```python
     import os
     from llama_index.llms.openai import OpenAI

     llm = OpenAI(model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"])
     ```

2. **Set Up the Embedding Model**:
   - Similarly, set up the embedding model using OpenAI's text-embedding-3-small model.
     ```python
     from llama_index.embeddings.openai import OpenAIEmbedding

     embed_model = OpenAIEmbedding(model="text-embedding-3-small")
     ```

3. **Configure Global Settings**:
   - Make the LLM and embedding models global for all LlamaIndex functions to use by setting them as default in the settings.
     ```python
     from llama_index.core import Settings

     Settings.llm = llm
     Settings.embed_model = embed_model
     ```

4. **Load Data into DuckDB**:
   - Load your data into DuckDB. For example, you can load PDF files from a directory and convert them into embeddings to store in a vector store.
     ```python
     from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
     from llama_index.vector_stores.duckdb import DuckDBVectorStore

     documents = SimpleDirectoryReader("Data").load_data()
     ```

5. **Create a Vector Store**:
   - Create a vector store in DuckDB to store the embeddings. This will allow you to perform efficient similarity searches.
     ```python
     # Assuming you have a database called "datacamp.duckdb"
     vector_store = DuckDBVectorStore("datacamp.duckdb")
     ```

6. **Integrate with LLM**:
   - Use the LLM to generate responses based on queries. You can use the vector store to retrieve relevant documents and pass them to the LLM for generating context-aware responses.
     ```python
     response = chat_engine.chat("Your query here")
     ```

7. **Create a Memory Buffer**:
   - To create a memory buffer, you can store the context of previous interactions in a data structure (e.g., a list or a database table) and use it to provide context for future queries. This allows the LLM to remember past interactions and provide more coherent responses.

These steps outline the process of integrating DuckDB with an LLM and creating a memory buffer. The key is to use the vector store for efficient retrieval and maintain a context of interactions to enhance the LLM's responses.

In [78]:
con.close()

In [79]:
from sqlalchemy import create_engine

engine = create_engine("duckdb:///database_for_testing_duck.duckdb")
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM bank LIMIT 3")
    print(cursor.fetchall())

[(0, 56, 'housemaid', 'married', 'basic.4y', 'no', 'no', 'no', 'telephone', 'may', 'mon', 261, 1, 999, 0, 'nonexistent', None, None, None, 4.857, None, False), (1, 57, 'services', 'married', 'high.school', 'unknown', 'no', 'no', 'telephone', 'may', 'mon', 149, 1, 999, 0, 'nonexistent', None, None, None, 4.857, None, False), (2, 37, 'services', 'married', 'high.school', 'no', 'yes', 'no', 'telephone', 'may', 'mon', 226, 1, 999, 0, 'nonexistent', None, None, None, 4.857, None, False)]


In [80]:
from llama_index.core import SQLDatabase
sql_database = SQLDatabase(engine, include_tables=["bank"])

/Users/emanuelalves/Library/Python/3.9/lib/python/site-packages/duckdb_engine/__init__.py:174: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [81]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(sql_database)

In [82]:
response = query_engine.query("Which is the longest running campaign?")

print(response.response)

The longest running campaign is campaign number 1, with a duration of 1666 days.


In [83]:
response = query_engine.query("Which type of job has the most housing loan?")
print(response.response)

The type of job with the most housing loans is "blue-collar," with a total of 12 individuals having housing loans.


In [84]:
print(response.metadata)

{'c53146f2-6b09-42fb-89df-e6335336ca64': {'sql_query': "SELECT job, COUNT(housing) AS housing_loan_count FROM bank WHERE housing = 'yes' GROUP BY job ORDER BY housing_loan_count DESC LIMIT 1", 'result': [('blue-collar', 12)], 'col_keys': ['job', 'housing_loan_count']}, 'sql_query': "SELECT job, COUNT(housing) AS housing_loan_count FROM bank WHERE housing = 'yes' GROUP BY job ORDER BY housing_loan_count DESC LIMIT 1", 'result': [('blue-collar', 12)], 'col_keys': ['job', 'housing_loan_count']}


In [86]:
engine.close()

AttributeError: 'Engine' object has no attribute 'close'